In [ ]:
#Memory buffer, is unresponsive
import cv2
import mediapipe as mp
import requests
import numpy as np
import imutils

# Initialize MediaPipe Hand module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Replace the URL with your Android camera's IP URL
url = "http://192.168.1.17:8080/shot.jpg"

# Define the "Thumbs Up" gesture using landmark indices
thumbs_up_indices = [1, 2, 3, 4]

# Initialize a variable to track the detected gesture
detected_gesture = None

# While loop to continuously fetch data from the URL
while True:
    # Retrieve the image from the Android camera
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)

    # Process the image with MediaPipe Hand module
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks on the frame
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Check for the "Thumbs Up" gesture
                is_thumbs_up = all(landmarks[i].y < landmarks[i + 1].y for i in thumbs_up_indices)
                if is_thumbs_up:
                    detected_gesture = "Thumbs Up"
                else:
                    detected_gesture = None

                # Display landmark indices as numbers
                for i, landmark in enumerate(hand_landmarks.landmark):
                    x, y = int(landmark.x * img.shape[1]), int(landmark.y * img.shape[0])
                    cv2.putText(img, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the processed image with hand landmarks, numbers, and detected gesture
    cv2.putText(img, f"Detected Gesture: {detected_gesture}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Android_cam", img)

    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
